# Detecting whether pairs of genomes and contaminants occur more than would be expected by chance

## Idea

Can we detect whether pairs of contaminants occur more often than would be expected by chance (like are the majority of *F. prausnitzii* genomes contaminated with it’s friend *R. hominis*?). 

## Approach

1. **Estimate the liklihood that a contaminant occurs by chance.** For each species in GTDB, calculate the fraction of genomes in GTDB that are contaminated with that species. This will be the baseline proportion for that species occuring as a contaminant by chance.
2. **Estimate the observed proportion of genomes in each species that are contaminated with every other species.** 
3. **Test whether a species-contaminant pair occurs more often than would expected by chance.** Use a one (two?)-sample test of proportion to see whether the observed proportion of genomes within a species that are contaminated by a species is higher than was estimated to occur by chance.

## Edit -- probably a better approach:

+ Use co-occurence libraries from microbiology/ecology

## Thinking through input files

Does the information needed to do this readily exists, ideally from stage1 alone, or does it need to be created?

**From stage1 outputs:**

It looks like I could do this at the order level using either `*.contam_summary.json` or `*.contigs-tax.json`, which record the order of each contig within a genome.
I could post-process the `.matches.csv` file, which includes the genome name and its similarity to all genomes in the database, summarizing it to taxonomy and using this to identify species-species relationships. I think the `matches.csv` is the prefetch results though, so it would only be presence/absence of species, not proportion of overlap, given the potential and likelihood of double counting

**From stage2 outputs:**

I wasn’t really planning on making stage2 outputs for all of GTDB, so I’d rather not use these, but if I did, I could use `*hitlist.matches.yaml`
 or `*.matches.json`, which both record contaminant genome identifiers, lineages, and counts. I guess I could make stage2 outputs only up until the `@toplevel` `rule hitlist_make_contigs_matches` to make these files, which would avoid mashmap and a lot of the other “heavy lifting” in stage2.

In [1]:
setwd("..")

In [3]:
library(jsonlite)
library(purrr)
library(dplyr)
library(tidyr)
library(tibble)
library(cooccur)
library(visNetwork)
library(igraph)

## Order-level co-occurrence analysis

Adapted from tutorial at https://medium.com/analytics-vidhya/how-to-create-co-occurrence-networks-with-the-r-packages-cooccur-and-visnetwork-f6e1ceb1c523

In [4]:
read_contigs_tax <- function(contigs_tax_path){  
  json <- fromJSON(contigs_tax_path)
  contig_tax_all <- data.frame()
  for(i in 1:length(json)){
    contig_name <- names(json)[i]
    basepairs <- json[[i]][[1]]
    hashes <- json[[i]][[2]]
    if(length(json[[i]][[3]]) > 0){
      lineage <- json[[i]][[3]][[1]][[1]][,2]
      lineage = paste(lineage, collapse = ";", sep = ";")
      matched_hashes <- json[[i]][[3]][[1]][[2]]
    } else {
      lineage = NA
      matched_hashes = NA
    }
    contig_tax <- data.frame(contig_name, basepairs, hashes, lineage, matched_hashes)
    contig_tax_all <- bind_rows(contig_tax_all, contig_tax)
  }
   contig_tax_all$genome <- gsub(".contigs-tax.json", "", basename(contigs_tax_path))
   return(contig_tax_all)
}

In [5]:
# run as a for loop so empty jsons can be skipped.
files <- Sys.glob("outputs/gtdb_rs202_charcoal1/stage1/*contigs-tax.json")
populated_files <- character()
for(i in 1:length(files)){
  file <- files[i]
  json <- fromJSON(file)
  if(length(json) > 0){
      populated_files <- c(populated_files, file)
  }
}

In [6]:
length(populated_files)
length(files)

[1] 12808

[1] 12921

In [7]:
contigs_tax <- populated_files %>%
  map_dfr(read_contigs_tax)

In [8]:
contigs_tax <- contigs_tax %>% 
  group_by(genome, lineage) %>%
  summarize(matched_hashes = sum(matched_hashes)) 

`summarise()` has grouped output by 'genome'. You can override using the `.groups` argument.



In [9]:
# put genome across the top, lineages as rows
contigs_tax_pa <- contigs_tax %>%
  pivot_wider(id_cols = lineage, names_from = "genome", values_from = "matched_hashes") %>%
  filter(!is.na(lineage)) %>%
  column_to_rownames("lineage") %>%
  replace(is.na(.), 0) %>%
  mutate_if(is.numeric, ~1 * (. > 0))


In [10]:
contigs_tax_pa

,GCA_000018925.1_genomic.fna.gz,GCA_000143435.1_genomic.fna.gz,GCA_000155165.1_genomic.fna.gz,GCA_000172255.1_genomic.fna.gz,GCA_000184515.2_genomic.fna.gz,GCA_000186985.3_genomic.fna.gz,GCA_000188815.2_genomic.fna.gz,GCA_000221785.2_genomic.fna.gz,GCA_000232345.2_genomic.fna.gz,GCA_000232685.2_genomic.fna.gz,⋯,GCF_903959745.1_genomic.fna.gz,GCF_903969835.1_genomic.fna.gz,GCF_903970045.1_genomic.fna.gz,GCF_903970095.1_genomic.fna.gz,GCF_903970905.1_genomic.fna.gz,GCF_903986205.1_genomic.fna.gz,GCF_903986375.1_genomic.fna.gz,GCF_903986485.1_genomic.fna.gz,GCF_903986905.1_genomic.fna.gz,GCF_903989505.1_genomic.fna.gz
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Francisellales,1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales,0,1,0,0,0,0,0,0,1,1,⋯,0,1,0,0,0,0,0,0,0,0
d__Bacteria;p__Actinobacteriota;c__Actinomycetia;o__Mycobacteriales,0,0,1,0,0,0,1,0,0,0,⋯,0,0,0,0,0,1,1,1,0,0
d__Bacteria;p__Spirochaetota;c__Spirochaetia;o__Borreliales,0,0,0,1,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales,0,0,0,0,1,0,1,0,0,0,⋯,0,0,0,1,1,0,0,0,1,0
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pseudomonadales,0,0,0,0,1,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
d__Bacteria;p__Firmicutes;c__Bacilli;o__Mycoplasmatales,0,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
d__Bacteria;p__Firmicutes;c__Bacilli;o__Staphylococcales,0,0,0,0,0,0,0,1,0,0,⋯,0,1,1,1,0,0,0,0,0,0
d__Bacteria;p__Campylobacterota;c__Campylobacteria;o__Campylobacterales,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [11]:
co <- print(cooccur(contigs_tax_pa, spp_names = TRUE))

- **sp1**: numeric indices for species 1
- **sp2**: numeric indices for species 2
- **p_lt**: if p_lt < 0.05, then the species pair co-occurs at a frequency lower than we would expect to find by chance
- **p_gt**: if p_gt < 0.05, the pair co-occurs at a rate higher than we would expect to find by chance

## Summary

Approach does not scale well to 10k datasets, so probably will scale even less to 60k or 350k. Will look for new approaches